# Aufgabe 1: Thermoformen in der Praxis
## Lies dir die Seiten 34 und 35 des Buches "Thermoformen in der Praxis" sorgfältig durch. Erstelle anschließend eine Funktion, die die Dimensionen einer Kavität als Eingabe erhält und die praktisch erreichbare Wanddickenverteilung berechnet. Runde das Ergebnis auf eine Nachkommastelle.

## Angaben:

### Kavität: Länge x Breite x Höhe = 150 x 150 x 30 mm
### Grundfläche: Länge x Breite = 180 x 180 mm
### Ausgangsmaterialstärke: 2 mm

In [9]:
def wall_thickness_distribution(base_length,base_width,cavity_length, cavity_width, cavity_heigth, initial_thickness):
    F1 = base_length * base_width
    F2 = F1 + 2*cavity_width*cavity_heigth + 2*cavity_length*cavity_heigth

    average_thickness = round(initial_thickness * (F1 / F2), 1)

    return average_thickness


In [10]:
wall_thickness_distribution(180,180,150,150,30,2)

1.3

# Aufgabe 2: Berechnung der Durchbiegung eines Trays

## Lies dir die Seiten 502 und 457 des Buches "Roark's Formulas for Stress and Strain" aufmerksam durch. Entwickle anschließend eine Funktion, die die Durchbiegung eines Trays berechnet. Verwende dabei die Angaben der Grundfläche und der erreichten Wanddickenverteilung als Materialstärke aus Aufgabe 1. Gehe davon aus, dass ein Gewicht von 1 kg gleichmäßig auf die Platte wirkt. Die Materialart ist PS-HI (Polystyrol-High Impact), welches ein E-Modul von 2150 N/mm² besitzt.

In [175]:
from scipy.interpolate import interp1d


def get_alpha_func():
    alpha = [0.0444, 0.0616, 0.0770, 0.0906, 0.1017, 0.1110, 0.1335, 0.1400, 0.1417]
    ratio = [1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 3.0, 4.0, 5.0]

    f = interp1d(ratio, alpha)

    return f

def get_alpha(ratio):

    alpha_func = get_alpha_func()

    if ratio <= 5:
        alpha = alpha_func(ratio)
    else:
        alpha = 0.1421

    return alpha

def get_force_per_unit_area(weight, base_length, base_width):
    gravity_constant=9.81
    force = weight * gravity_constant
    force_per_unit_area = force / (base_length * base_width)
    return force_per_unit_area



#W = total applied load (force) N
#q = load per unit area : q = W / (base_length * base_width) : N/mm2
#y = vertical deflection of plate (length) mm
#E = modulus of elasticity (force per unit area) N/mm2
#t = plate thickness mm2
#alpha = a constant that depends on the aspect ratio of the plate
# y = -alpha * q/(a*b) * (b^4) / (E * (t^3))

def deflection(base_length, base_width, thickness, materialtyp, weight):

    #modulus = {"PS-HI": 2150}
    #elastic_modulus = modulus[materialtyp]
    elastic_modulus = 2150 # Hardcoded auch ok

    dims = [base_length, base_width]
    dims.sort()

    ratio = dims[1] / dims[0]

    alpha = get_alpha(ratio)
    force_per_unit_area = get_force_per_unit_area(weight, base_length, base_width)

    deflection = (-alpha* force_per_unit_area * (dims[0]**4))/(elastic_modulus*(thickness**3))

    return deflection




    


    


In [180]:
deflection(180, 180, 1.3, "PS-HI",1)

-2.9876414137671876

# Aufgabe 3: Berechnung der Ausgangsmaterialstärke basierend auf der maximalen Durchbiegung
## Verbinde die Aufgabenstellungen der vorherigen beiden Aufgaben und entwickle eine Funktion, die ausgehend von einer gewünschten maximalen Durchbiegung des Trays die erforderliche Ausgangsmaterialstärke berechnet. Berechne die notwendige Ausgangsmaterialstärke für eine maximale Durchbiegung von 1 mm unter ansonsten gleichen Bedingungen/Angaben wie in Aufgabe 1 und 2.

In [170]:
def initial_material_thickness(deflection, base_length, base_width, cavity_length, cavity_width, cavity_heigth, materialtyp, weight):

    F1 = base_length * base_width
    F2 = F1 + 2*cavity_width*cavity_heigth + 2*cavity_length*cavity_heigth

    thickness_ratio = F2 / F1

    #modulus = {"PS-HI": 2150}
    #elastic_modulus = modulus[materialtyp]
    elastic_modulus = 2150 # Hardcoded auch ok

    dims = [base_length, base_width]
    dims.sort()

    ratio = dims[1] / dims[0]

    alpha = get_alpha(ratio)
    force_per_unit_area = get_force_per_unit_area(weight, base_length, base_width)

    thickness = ((-alpha * force_per_unit_area * (dims[0]**4)) / (elastic_modulus * deflection))**(1/3)*thickness_ratio

    return thickness

In [166]:
# Überprüfung der Funktionen
initial_material_thickness(-2.9876414137671876, 180, 180, 150, 150, 30, "PS-HI", 1)

2.022222222222222

In [171]:
initial_material_thickness(-1, 180, 180, 150, 150, 30, "PS-HI", 1)

2.9125386829690068